<a href="https://colab.research.google.com/github/adnanowaiss/hello-world/blob/master/Neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [19]:
# download training data from the FashionMNISTdataset.
training_data = datasets.FashionMNIST(
    train=True,
    transform=ToTensor(),
    download=True,
    root="data"
)

In [20]:
# download test data from the FashionMNIST dataset.
test_data = datasets.FashionMNIST(
    train=False,
    transform=ToTensor(),
    download=True,
    root="data"
)

In [21]:
batch_size = 64

# create data loaders
training_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=64)

for X, y in test_loader:
  print(f"Shape of X [N C H W]: {X.shape}")
  print(f"Shape of y: {y.shape} {y.dtype}")
  break

Shape of X [N C H W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [22]:
# get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.Flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
      x = self.Flatten(x)
      logits = self.linear_relu_stack(x)
      return logits

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (Flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [25]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [26]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(f"size: {size}")
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)  # Move input data to the device (GPU or CPU)
        y = y.to(device)  # Move target labels to the device (GPU or CPU)
        # compute predicted y by passing X to the model
        prediction = model(X)
        # compute the loss
        loss = loss_fn(prediction, y)
        #  apply zero gradients, perform a backward pass, and update the weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print training progress
        if batch % 100 == 0:
            loss_value = loss.item()
            current = batch * len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")


In [27]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            prediction = model(X)
            test_loss += loss_fn(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [28]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(training_loader, model, loss_fn, optimizer)
  test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
size: 60000
loss: 2.312397  [    0/60000]
loss: 2.205643  [ 6400/60000]
loss: 1.903807  [12800/60000]
loss: 1.603799  [19200/60000]
loss: 1.243132  [25600/60000]
loss: 1.125847  [32000/60000]
loss: 1.035178  [38400/60000]
loss: 0.902477  [44800/60000]
loss: 0.902463  [51200/60000]
loss: 0.823365  [57600/60000]
Test Error: 
 Accuracy: 69.8%, Avg loss: 0.815316 

Epoch 2
-------------------------------
size: 60000
loss: 0.830619  [    0/60000]
loss: 0.867745  [ 6400/60000]
loss: 0.600478  [12800/60000]
loss: 0.818005  [19200/60000]
loss: 0.688237  [25600/60000]
loss: 0.649418  [32000/60000]
loss: 0.708219  [38400/60000]
loss: 0.691370  [44800/60000]
loss: 0.706620  [51200/60000]
loss: 0.643141  [57600/60000]
Test Error: 
 Accuracy: 78.0%, Avg loss: 0.638265 

Epoch 3
-------------------------------
size: 60000
loss: 0.575935  [    0/60000]
loss: 0.664619  [ 6400/60000]
loss: 0.434282  [12800/60000]
loss: 0.690005  [19200/60000]
loss: 0.603553  [256

In [29]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [30]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# set model to evaluation mode
model.eval()

sample_index = 1  # sample Index (Change this index to select a different sample)
x, y = test_data[sample_index][0], test_data[sample_index][1]

# make prediction without gradient calculation
with torch.no_grad():
    x = x.to(device)
    prediction = model(x.unsqueeze(0))

    # get predicted and actual classes
    predicted, actual = classes[prediction.argmax(dim=1).item()], classes[y]

    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Pullover", Actual: "Pullover"
